In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import re
import matplotlib.pyplot as plt
from string import punctuation
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#print long strings without truncating in pandas
pd.options.display.max_colwidth = 500

max_len_bert = 512


In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
movie_dataset = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
clothing_dataset = pd.read_csv("../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv", index_col = 0)
food_dataset = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv", index_col =[0])

In [ ]:
clothing_dataset.head()

In [ ]:
movie_dataset.head()

In [ ]:
food_dataset.head()

Lets take a look at each of these datasets one at a time 

MOVIE DATASET:

In [ ]:
print("The shape of the movie dataset is: {}".format (movie_dataset.shape))
print('\nSome sample datapoint:\n')

movie_dataset.sample(5)

In [ ]:
#Lets get how balanced our movie dataset is
movie_dataset.sentiment.value_counts(normalize = True)

Our movie dataset is equally balanced. 

In [ ]:
#Do we have any nans?
#which columns have nans
movie_dataset.isnull().sum()

In [ ]:
#Lets convert the sentiments from categorical to numerical features
le = LabelEncoder()
movie_dataset['sentiment'] = le.fit_transform(movie_dataset['sentiment'])
movie_dataset.rename(columns = {'review':'Text', 'sentiment':'ratings'}, inplace=True)

movie_dataset.head()

Women E-ommerce clothing dataset

In [ ]:
print("The shape of the women dataset is: {}".format (clothing_dataset.shape))
print('\nSome sample datapoint:\n')

clothing_dataset.sample(5)

In [ ]:
#WE are interested in only 2 columns in this dataset. So we will drop some of the columns

clothing_dataset.columns

In [ ]:
clothing_dataset.info()

In [ ]:
clothing_dataset.drop([col for col in clothing_dataset.columns if col not in ['Review Text', 'Rating']], axis=1, inplace=True)


In [ ]:
clothing_dataset.head(3)

In [ ]:
clothing_dataset.Rating.unique()

In [ ]:
#Do we have any null/empty cell
#which columns have nans
clothing_dataset.isnull().sum()


In [ ]:
clothing_dataset.shape

In [ ]:
#Dropping all the empty/ null cell from the clothing dataset
clothing_dataset = clothing_dataset[~clothing_dataset['Review Text'].isnull()]

clothing_dataset.shape

In [ ]:
#we shall consider all ratings 1, 2 as negative reviews and reviews 4, 5 as positive reviews
#All reviews with Rating = 3 shall be considered neutrl reviews and for simplicity, shall be dropped.

clothing_reviews=clothing_dataset[clothing_dataset.Rating!=3]
clothing_reviews["ratings"]= clothing_dataset["Rating"].apply(lambda x: 1 if x > 3  else 0)
clothing_reviews.head()

In [ ]:
clothing_reviews=clothing_reviews.drop("Rating",axis=1)
clothing_reviews.rename(columns = {'Review Text':'Text'}, inplace=True)

clothing_reviews.head()

In [ ]:
clothing_reviews.ratings.value_counts(normalize = True)
# this dataset is very imbalanced

In [ ]:
sns.set(style = 'darkgrid')
ax = sns.countplot(x = 'ratings', data = clothing_reviews)
ax.set_title('Ratings distribution for women clothing dataset');

Amazon fine food dataset

In [ ]:
food_dataset.head()

In [ ]:
food_dataset.drop([col for col in food_dataset.columns if col not in ['Text','Score']], axis=1, inplace=True)
food_dataset.head()

In [ ]:
food_dataset.Score.unique()

In [ ]:
food_dataset.Score.value_counts(normalize = True)

*Since i want to have a binary classification problem with only positive and negative reviews, I will focus mostly on the reviews with 1, 2 as my negative reviews and the reviews with 4, 5 as my positive reviews.For simplicity,  I will dropp all reviews with 3 as i consider them as neutral reviews. *

In [ ]:
#we shall consider all ratings 1, 2 as negative reviews and reviews 4, 5 as positive reviews

food_reviews=food_dataset[food_dataset.Score!=3]
food_reviews["ratings"]= food_dataset['Score'].apply(lambda x: 1 if x > 3  else 0)
food_reviews=food_reviews.drop("Score",axis=1)

food_reviews.head()

In [ ]:
food_reviews.ratings.value_counts(normalize = True)


this dataset is highly imbalanced. But we will not worrry about it now. We will have to extract an equal number of positive and negative reviews when we are creating our unified dataset.

In [ ]:
#Lets find some empty cell and delete them
food_reviews.isnull().sum()

MULTICLASS DATASET CREATION

Description:

The main aim here was to create a unified dataset(multiclass_customer_review_data), which contains the movie_dataset, the women_dataset and the yelp_dataset into one dataset.

We attributed the

--- label '0' to the negative examples of all the 3 dataset.

--- label '1' for the positive examples from the movie_dataset

--- label '2' for the positive examples form the women_dataset

--- label '3' for the positive examples form the food_dataset

Since we are aiming for a balanced dataset, we are going to creat a balanced dataset for all the 3 datasets we are using, before creating the unified multiclass dataset. 

In [ ]:
#Lets get all the positive reviewss together

positive_movies_reviews = movie_dataset[movie_dataset.ratings == 1]

positive_clothing_reviews = clothing_reviews[clothing_reviews.ratings == 1]
positive_clothing_reviews['ratings'].replace(1, 2, inplace = True)

positive_food_reviews = food_reviews[food_reviews.ratings == 1]
positive_food_reviews['ratings'].replace(1, 3, inplace = True)


In [ ]:
#creating a new column to keep track of the originals in the new datasets

positive_movies_reviews['Originally'] = 'Movies'
positive_clothing_reviews['Originally'] = 'Clothing'
positive_food_reviews['Originally'] = 'Food' 

positive_movies_reviews.sample(10)

In [ ]:
#we get all the negative reviews together
negative_movies_reviews = movie_dataset[movie_dataset.ratings == 0]
negative_clothing_reviews = clothing_reviews[clothing_reviews.ratings == 0]
negative_food_reviews = food_reviews[food_reviews.ratings == 0]



In [ ]:
#creating a new column to keep track of their original datasets

negative_movies_reviews['Originally'] = 'Movies'

negative_clothing_reviews['Originally'] = 'Clothing'

negative_food_reviews['Originally'] = 'Food'

negative_movies_reviews.sample(10)

# Creating our dataset

In [ ]:
positive_dataset = pd.concat([positive_movies_reviews, positive_clothing_reviews, positive_food_reviews], axis = 0, ignore_index = True, sort= False)
positive_dataset.sample(10)


In [ ]:
#Dataframes were merged/concatenated correctly. We have no duplicate index.
positive_dataset.index[positive_dataset.index.duplicated()].unique()

In [ ]:
positive_dataset.shape

In [ ]:
def count_values(df, feature):
    total=df.loc[:,feature].value_counts()
    percentage=round(df.loc[:,feature].value_counts(normalize = True)*100, 2)
    
    return pd.concat([total, percentage], axis =1, keys = ['Total', 'Percentage'])

In [ ]:
count_values(positive_dataset, 'ratings')

In [ ]:
negative_dataset = pd.concat([negative_movies_reviews, negative_clothing_reviews, negative_food_reviews], axis=0, ignore_index = True, sort = False)

#negative_dataset.sample(10)

In [ ]:
negative_dataset.index[negative_dataset.index.duplicated()].unique()

In [ ]:
negative_dataset.shape

In [ ]:
negative_dataset.index[negative_dataset.index.duplicated()].unique()

In [ ]:
balanced_pos = positive_dataset[positive_dataset['Originally']=='Food'][:65000].append(positive_dataset[positive_dataset['Originally']=='Movies'], ignore_index = True, sort = False)
balanced_pos = balanced_pos.append(positive_dataset[positive_dataset['Originally']=='Clothing'], ignore_index = True, sort = False)
balanced_pos.shape

In [ ]:
balanced_pos.index[balanced_pos.index.duplicated()].unique()

In [ ]:
#Concatenating the positive and negative exxamples to form our multiclass dataset

#Concatenating the positive and the negative training datasets together

multiclass_dataset = pd.concat([negative_dataset, balanced_pos],ignore_index = True, sort = False)
#multiclass_dataset.sample(frac=1)#trying to shuffle it a little bit

multiclass_dataset.head(10)

In [ ]:
'''def shuffle(df, n=1, axis=0):     
    df = df.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df'''

In [ ]:
#Shuffle Pandas data frame
import sklearn.utils
multiclass_dataset = sklearn.utils.shuffle(multiclass_dataset)
multiclass_dataset = multiclass_dataset.reset_index(drop= True)
multiclass_dataset.head(10)

In [ ]:
multiclass_dataset.ratings.value_counts(normalize = True)

We see how ur unified dataset is distributed

In [ ]:
#Visualisation library
from plotly.offline import iplot
import plotly.graph_objs as go
#import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')


multiclass_dataset['ratings'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentages', 
                                                      linecolor='black', 
                                                      opacity=0.9,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.6,
                                                      gridcolor='white',
                                                     
                                                      title='Distribution of Ratings column in the multiclass dataset')

In [ ]:
multiclass_dataset.shape

# Text Preprocessing - preparation for Training with BILSTM

In [ ]:
multiclass_lstm = multiclass_dataset.copy()

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist

In [ ]:
err1 = multiclass_dataset['Text'].str.extractall("(&amp)")
err2 = multiclass_dataset['Text'].str.extractall("(\xa0)")

print('with &amp',len(err1[~err1.isna()]))
print('with (\xa0)',len(err2[~err2.isna()]))

In [ ]:
multiclass_dataset['Text'] = multiclass_dataset['Text'].str.replace('(&amp)','')
multiclass_dataset['Text'] = multiclass_dataset['Text'].str.replace('(\xa0)','')

In [ ]:
#https://www.kaggle.com/shirellamosi/sentiment-analysis-nlp
#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

STOPWORDS = set(stopwords.words('english'))
from string import punctuation
from bs4 import BeautifulSoup

def clean_text(text):
    '''
    Preprocess/normalize text
    '''
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    email_regex = '(^[a-z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)'
    happy_emoticon_regex = ': *\)'
    sad_emoticon_regex = ': *\('
    emojis ="🍕🐵😑😢🐶️😜😎👊🤪😁😍💖💵👎😀😂🔥⭐🤯😄🤪🏻💥😋👏😱🚌ᴵ͞🌟😊😳😧🍕🙀😐😕👍😮😃😘💩💯⛽🚄😖🏼🚲😟😈💪🙏🎯🌹😇💔😡👌🙄😠😉😤⛺🙂😏🍾🎉😞🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪🐰🐇🐱🙆😨🙃💕💗💚🙈😴🏿🤗🇺🇸⤵🏆🎃😩👮💙🐾🐕😆🌠🐟💫💰💎🖐🙅⛲🍰🤐👆🙌💛🙁👀🙊🙉🚬🤓😵😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓😣⏺😌🤑🌏😯😲💞🚓🔔📚🏀👐💤🍇🏡❔⁉👠》🇹🇼🌸🌞🎲😛💋💀🎄💜🤢َِ🗑💃📣👿༼つ༽😰🤣🐝🎅🍺🎵🌎͟🤡🤥😬🤧🚀🤴😝💨🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦🍀😫🤤🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪🇫🇷🇩🇪😷🇨🇦🌐📺🐋💘💓💐🌋🌄🌅👺🐷🚶🤘ͦ💸👂👃🎫🚢🚂🏃👽😙🎾👹⎌🏒⛸🏄🐀🚑🤷🤙🐒🐈ﷻ🦄🚗🐳👇⛷👋🦊🐽🎻🎹⛓🏹🍷🦆♾🎸🤕🤒⛑🎁🏝🦁🙋😶🔫👁💲🗯👑🚿💡😦🏐🇰🇵👾🐄🎈🔨🐎🤞🐸💟🎰🌝🛳🍭👣🏉💭🎥🐴👨🤳🦍🍩😗🏂👳🍗🕉🐲🍒🐑⏰💊🌤🍊🔹🤚🍎𝑷🐂💅💢💒🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻🤖🎎😼🕷👼📉🍟🍦🌈🔭《🐊🐍🐦🐡💳ἱ🙇🥜🔼"

    #number_regex = 'num'
    white_space_regex = '\s+'
    extra_white_space_regex = '^\s+|\s+$'
    lemmatizer = WordNetLemmatizer()
    
    text = text.lower()  #lower casing our text
    
    words = re.split('\s+', text)
    
    for w in range(len(words)):
       if re.match(email_regex, words[w]): 
           words[w] = ' email ' #converting all emails 
       elif re.match(url_regex, words[w]):
           words[w] = ' url '#converting all urls
    
    #removing english stopwords
    word = [word for word in text.split(" ") if word not in STOPWORDS] # removing stop words
    
    #words = [word for word in text.split(" ") if word not in punctuation]
    
    #lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    text = ' '.join(words)
    
    #Emogies
    '''text = re.sub(happy_emoticon_regex, ' happyemot ', text)
    text = re.sub(sad_emoticon_regex, ' sademot ', text)
    '''
    for emoji in emojis:
        text = text.replace(emoji, '')
    
    text = BeautifulSoup(text, 'lxml').get_text()   # removing html tags
        
    text = re.sub('\*+', '\*', text) # reduce scurbbing marker to one occurrence
    
    text = re.sub('\<br />', ' ', text)
    
    #Spacings
    text = re.sub(white_space_regex, ' ', text)
    text = re.sub(extra_white_space_regex, '', text)
    

    return text

In [ ]:
# Applying the cleaning function to our datasets
multiclass_lstm['clean_text'] = multiclass_lstm['Text'].apply(str).apply(lambda x: clean_text(x))
multiclass_lstm.head()

I will not remove numbers form my text. I think this nubers may be useful in training my model to recognise reviews. For example, reviews with body measurments mmay indicate that we are dealing with a clothing review.

# Removing all punctuations

In [ ]:
def punctuation_removal(messy_str):
    clean_list = [char for char in messy_str if char not in punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

In [ ]:
multiclass_lstm['clean_text'] = multiclass_lstm['clean_text'].apply(str).apply(lambda x: punctuation_removal(x))


In [ ]:
multiclass_lstm.sample(6)

# Decontract

In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

In [ ]:
multiclass_lstm['clean_text'] = multiclass_lstm['clean_text'].apply(str).apply(lambda x: decontract(x))

In [ ]:
#Distribution of words in the target text
classes = ['Food', 'Movies', 'Clothing']
fig, arr = plt.subplots(1, 3, figsize = (18,5))

color= ['blue', 'green', 'red']

for i in range(3):

    multiclass_lstm[multiclass_lstm['Originally'] == classes[i]]['clean_text'].str.split().str.len().plot.hist(ax = arr[i], color = color[i])
    arr[i].set_title(classes[i], color = color[i]);
    
fig.suptitle('Distrfibution of Number of words in cleaned text', fontsize = 18);

- About 140000 food reviews have about 100 - 200 words
- Abiut 40000 movies reviews have about 100 words

We find the longest sentence to determine our max length 

In [ ]:
# Max Length
# calculating length of the longest text to help in sequence padding

multiclass_lstm['review_len'] = multiclass_lstm['clean_text'].apply(len)
print(multiclass_lstm['review_len'].describe())

print('\n\nThe Longest sentence has Maximum length: ', max(multiclass_lstm['review_len']))

#Checkout the mean review length

In [ ]:
multiclass_lstm[multiclass_lstm['clean_text'].isnull()].shape

# Splitting dataset into training, validation and testing set

In [ ]:
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
np.random.seed(7)

#Splitting into training and testing set in the ration 80:20

train_DF, test_DF = train_test_split(multiclass_lstm, test_size = 0.2, random_state = 42)  #We get the testing set


print("Training data size : ", train_DF.shape)
print("Test data size : ", test_DF.shape)

To avoid contamination, we will output the current testing set so it is never touched



In [ ]:
# test_df.to_csv('my_test_file.csv',index=False)
# train_df.to_csv("my_train_set.csv", index = False)

# Loading the train dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
#Loading the train dataset
train_df = pd.read_csv("../input/multiclass-train/my_train_set.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
def sub_num(text):
    #substitutinging numbers
    text = re.sub('[0-9]+', 'numbr', text)
    return text

In [ ]:
train_df['clean_text'] = train_df['clean_text'].apply(str).apply(lambda x: sub_num(x))

In [ ]:
train_df.head()

In [ ]:
train_df['review_len'].describe()